In [1]:
import pandas as pd
from pathlib import Path
import os

In [3]:
directory = Path(os.getcwd())   # 'c:\\Users\\samue\\OneDrive\\Desktop\\JPM Fintech\\HKUST-x-J.P.-Morgan-Fintech-Mentorship-Program\\model_testing\\supervised'
directory = directory.parent.parent.parent.joinpath("data")  # 'c:\\Users\\samue\\OneDrive\\Desktop\\JPM Fintech\\data'
df = pd.read_pickle(directory.joinpath("stocktwits_raw_sample.pkl"))
df

,id,text,time,sentiment
0,417739155,$GOEV it’s so easy to see insiders buying befo...,1639643526,
1,417739053,$QQQ $AAPL $SPY $NDX Santa’s rally has an addr...,1639643425,
2,417738922,$AAPL Futures Ripping Means \nAAPL up 2% to 4%...,1639643308,
3,417738486,$AAPL pullback today,1639642896,Bearish
4,417738230,$AAPL look! The teddy bear picnic is starting!...,1639642665,Bullish
...,...,...,...,...
47815,419624841,$PYPL $V holding up well to the long side. $MC...,1640145912,
47816,419623909,$V $MSFT $MMM $HPQ suggested for $XMI US ARCA ...,1640145541,
47817,419618889,$SQ and $V similar charts. We will see $180,1640143905,
47818,419605709,$V damn putting in some work last couple of we...,1640139998,Bullish


In [6]:
import re
# given a text, find the related symbols and remove it 
# return: a string of text and a list of symbols
def find_and_remove_syb(text):
    symbols = re.findall(r"\B[$][a-zA-Z]*", text)
    symbols = [i for i in symbols if i != "$"]
    for symbol in symbols:
        text = text.replace(symbol, "")
    return text, symbols

# create a new dataframe
df_stocks = pd.DataFrame(columns=['text', 'time', 'symbols'])
for ind in range(len(df)):
    text, symbol = find_and_remove_syb(df.iloc[ind].text)
    if len(symbol) == 1:
        df_stocks = df_stocks.append({"text": text, "symbols": symbol[0], "time": df.iloc[ind].time}, ignore_index=True)
df_stocks

,text,time,symbols
0,pullback today,1639642896,$AAPL
1,look! The teddy bear picnic is starting! Lol,1639642665,$AAPL
2,,1639642565,$AAPL
3,classic inflation of calls in order to load u...,1639641515,$AAPL
4,"Guys, AAPL consolidating well and ready to go...",1639641435,$AAPL
...,...,...,...
36276,great day 👍👍,1640153385,$V
36277,inverse head n shoulders.... 222 possible tom...,1640149580,$V
36278,[15s. delayed]: Issued Press Release on Decem...,1640148346,$V
36279,damn putting in some work last couple of weeks..,1640139998,$V


In [10]:
def remove_website(text):
    for word in text.split():
        if 'http' in word:
            text = text.replace(word, '')
    return text

df_stocks['text'] = [remove_website(i) for i in list(df_stocks['text'])]

import html
df_stocks['text'] = [html.unescape(i) for i in list(df_stocks['text'])]

df_stocks['symbols'] = [i.upper() for i in df_stocks['symbols']]

In [12]:
df_stocks['symbols'].value_counts()

$TSLA    16052
$AAPL     5588
$PYPL     4177
$FB       3566
$MSFT     2699
$BAC       879
$V         805
$JPM       714
$JD        621
$MS        505
$GS        383
$KO        292
Name: symbols, dtype: int64

In [13]:
df_stocks.to_pickle(directory.joinpath("stocktwits_processed_without_multiple_sample.pkl"))